In [1]:
import sys
sys.path.append('../../Modules/')

from Packages import *
from Json_processor import *

In [2]:
main_df = ARC_Challenge_Processor('ARC-Challenge','ARC-Challenge-Test.jsonl')
main_df = main_df.convert_df()